<a href="https://colab.research.google.com/github/BruceXavierChou/stock_analysis/blob/main/%E3%80%8Cstk_ch05_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-05 AI 技術指標回測系統

## 5-2 強大的回測工具：backtesting.py

### 1️⃣ 安裝及匯入套件

In [ ]:
!pip install openai
!pip install yfinance
!pip install backtesting
!pip install bokeh==2.4.3 # 繪圖套件
from  openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import datetime as dt # 時間套件
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=2b223f2bdfbd8d70d16f71de460cda56c892d20c1fcdd933a3081874f3b5ad24
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 35.6 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.3.4
    Uninstalling bokeh-3.3.4:
      Successfully uninstalled bokeh-3.3.4
ERROR: pip's dependency res

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


### 2️⃣ 取得股價資料

In [ ]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
df = yf.download(stock_id, period="5y")
# 計算指標
df['ma1'] = df['Close'].rolling(window=5).mean()
df['ma2'] = df['Close'].rolling(window=10).mean()
df.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ma1,ma2
Date,,,,,,,,
2019-07-17,254.0,254.0,251.0,252.0,225.442749,25573648,NaN,NaN
2019-07-18,252.0,254.5,251.5,254.0,227.231964,20842575,NaN,NaN
2019-07-19,259.5,261.0,259.0,259.0,231.705048,59709798,NaN,NaN
2019-07-22,263.0,264.0,262.0,264.0,236.178116,31522639,NaN,NaN
2019-07-23,265.5,266.5,262.5,264.0,236.178116,29273223,258.6,NaN


### 3️⃣ 定義回測策略

In [ ]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
      self.buy(size=1)
    elif crossover(self.data.ma2, self.data.ma1):
      self.sell(size=1)

### 4️⃣ 回測結果

In [ ]:
backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()

# 印出回測績效
print(stats)

# 查看詳細的交易紀錄
stats["_trades"].head()

Start                     2019-07-17 00:00:00
End                       2024-07-17 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   78.553821
Equity Final [$]                    99498.656
Equity Peak [$]                    100466.314
Return [%]                          -0.501344
Buy & Hold Return [%]              308.730159
Return (Ann.) [%]                   -0.104019
Volatility (Ann.) [%]                0.215469
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.007958
Avg. Drawdown [%]                   -0.096393
Max. Drawdown Duration      631 days 00:00:00
Avg. Drawdown Duration       90 days 00:00:00
# Trades                                   64
Win Rate [%]                             37.5
Best Trade [%]                      35.820355
Worst Trade [%]                    -21.024639
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,12,24,250.992,254.5,-3.508,-0.013977,2019-08-02,2019-08-21,19 days
1,-1,30,33,251.988,256.5,-4.512,-0.017906,2019-08-29,2019-09-03,5 days
2,-1,67,68,293.820,297.0,-3.180,-0.010823,2019-10-28,2019-10-29,1 days
3,-1,80,84,303.282,314.0,-10.718,-0.035340,2019-11-14,2019-11-20,6 days
4,-1,90,97,308.760,314.0,-5.240,-0.016971,2019-11-28,2019-12-09,11 days


### 5️⃣ 回測繪圖

In [ ]:
backtest.plot(plot_equity=True,
       plot_return=False,
       plot_pl=True,
       plot_volume=True,
       plot_drawdown=False,
       superimpose=True)

Row(id='1407', ...)

### 6️⃣ 設定停利、停損點

In [ ]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
        # 買入時設置停損與停利價格
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.ma2, self.data.ma1):
        # 賣出時時設置停損與停利價格
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)

backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()
print(stats)

Start                     2019-07-17 00:00:00
End                       2024-07-17 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                     49.4659
Equity Final [$]                    99716.022
Equity Peak [$]                    100004.492
Return [%]                          -0.283978
Buy & Hold Return [%]              308.730159
Return (Ann.) [%]                   -0.058869
Volatility (Ann.) [%]                0.091964
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.381141
Avg. Drawdown [%]                    -0.12955
Max. Drawdown Duration     1800 days 00:00:00
Avg. Drawdown Duration      604 days 00:00:00
# Trades                                   70
Win Rate [%]                        27.142857
Best Trade [%]                      14.218451
Worst Trade [%]                    -12.260064
Avg. Trade [%]                    

## 5-3 讓 AI 產生回測策略

### 7️⃣ 輸入 OpenAI API KEY

In [ ]:
import getpass
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key=api_key)

請輸入金鑰：··········


### 8️⃣ 創建 GPT 3.5 模型函式

In [ ]:
# GPT 3.5 模型
def get_reply(messages):
  try:
    response = client.chat.completions.create(model="gpt-3.5-turbo",
                         messages=messages)
    reply = response.choices[0].message.content
  except OpenAIError as err:
    reply = f"發生 {err.type} 錯誤\n{err.message}"
  return reply

# 設定 AI 角色, 使其依據使用者需求進行 df 處理
def ai_helper(df, user_msg):

  msg = [{
    "role":
    "system",
    "content":
    f"As a professional code generation robot, \
      I require your assistance in generating Python code \
      based on specific user requirements. To proceed, \
      I will provide you with a dataframe (df) that follows the \
      format {df.columns}. Your task is to carefully analyze the \
      user's requirements and generate the Python code \
      accordingly.Please note that your response should solely \
      consist of the code itself, \
      and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg} \n\
      Your task is to develop a Python function named \
      'calculate(df)'. This function should accept a dataframe as \
      its parameter. Ensure that you only utilize the columns \
      present in the dataset, specifically {df.columns}.\
      After processing, the function should return the processed \
      dataframe. Your response should strictly contain the Python \
      code for the 'calculate(df)' function \
      and exclude any unrelated content."
  }]

  reply_data = get_reply(msg)
  return reply_data

# 產生技術指標策略
def ai_strategy(df, user_msg, add_msg="無"):

  code_example ='''
class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.short_ma, self.data.long_ma):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.long_ma, self.data.short_ma):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)
        '''

  msg = [{
    "role":
    "system",
    "content":
     "As a Python code generation bot, your task is to generate \
     code for a stock strategy based on user requirements and df. \
     Please note that your response should solely \
     consist of the code itself, \
     and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
     "The user requirement:計算 ma,\n\
     The additional requirement: 請設置 10% 的停利與停損點\n\
     The df.columns =['Open',	'High', 'Low',	'Close',	'Adj Close',	'Volume', 'short_ma',	'long_ma']\n\
     Please using the crossover() function in next(self)\
     Your response should strictly contain the Python \
     code for the 'AiStrategy(Strategy)' class \
     and exclude any unrelated content."
  }, {
    "role":
    "assistant",
    "content":f"{code_example}"
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg}\n\
     The additional requirement:{add_msg}\n\
     The df.columns ={df.columns}\n\
     Your task is to develop a Python class named \
     'AiStrategy(Strategy)'\
     Please using the crossover() function in next(self)."

  }]

  reply_data = get_reply(msg)
  return reply_data


### 9️⃣ 計算技術指標

In [ ]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
df = yf.download(stock_id, period="5y")
# 計算指標
user_msg = ["MACD", "請設置10%的停損點與20%的停利點"]
code_str = ai_helper(df, user_msg[0])
print(code_str)
exec(code_str)
new_df = calculate(df)
new_df.tail()

[*********************100%%**********************]  1 of 1 completed


def calculate(df):
    short_ema = df['Close'].ewm(span=12, adjust=False).mean()
    long_ema = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = short_ema - long_ema
    return df


,Open,High,Low,Close,Adj Close,Volume,MACD
Date,,,,,,,
2024-07-11,1065.0,1080.0,1055.0,1080.0,1080.0,41724336,43.947490
2024-07-12,1030.0,1045.0,1025.0,1040.0,1040.0,68983062,43.102706
2024-07-15,1040.0,1045.0,1025.0,1040.0,1040.0,38393309,41.949640
2024-07-16,1040.0,1070.0,1035.0,1055.0,1055.0,32125609,41.764763
2024-07-17,1035.0,1045.0,1020.0,1030.0,1030.0,57611197,39.149662


### 🔟 策略生成

In [ ]:
strategy_str = ai_strategy(new_df, user_msg[0], user_msg[1])
print(strategy_str)
print("-----------------------")
exec(strategy_str)
backtest = Backtest(df,
        AiStrategy,
        cash=100000,
        commission=0.004,
        trade_on_close=True,
        exclusive_orders=True,
        )
stats = backtest.run()
print(stats)

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, 0):
        self.buy(size=1,
                 sl=self.data.Close[-1] * 0.90,
                 tp=self.data.Close[-1] * 1.20)
    elif crossover(0, self.data.MACD):
        self.sell(size=1,
                  sl=self.data.Close[-1] * 1.10,
                  tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2019-07-17 00:00:00
End                       2024-07-17 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   73.623665
Equity Final [$]                   100223.496
Equity Peak [$]                    100229.996
Return [%]                           0.223496
Buy & Hold Return [%]              308.730159
Return (Ann.) [%]                    0.046238
Volatility (Ann.) [%]                0.109217
Sharpe Ratio                         0.423354
Sortino Ratio                        0.640277
Calmar Ratio       

### 1️⃣1️⃣ 寫成函式

In [ ]:
def ai_backtest(stock_id, period, user_msg, add_msg):

  # 下載資料
  df = yf.download(stock_id, period=period)

  # 獲取和執行指標計算程式碼
  code_str = ai_helper(df, user_msg)
  local_namespace = {}
  exec(code_str, globals(), local_namespace)
  calculate = local_namespace['calculate']
  new_df = calculate(df)

  # 獲取和執行策略程式碼
  strategy_str = ai_strategy(new_df, user_msg, add_msg)
  print(strategy_str)
  print("-----------------------")
  exec(strategy_str, globals(), local_namespace)
  AiStrategy = local_namespace['AiStrategy']

  backtest = Backtest(df,
          AiStrategy,
          cash=100000,
          commission=0.004,
          trade_on_close=True,
          exclusive_orders=True,
          )
  stats = backtest.run()
  print(stats)
  return str(stats)


## 5-4 讓 AI 解析回測報告

### 1️⃣2️⃣ 設定 AI 回復內容

In [ ]:
def backtest_analysis(*args):

  content_list = [f"策略{i+1}：{report}"
                  for i, report in enumerate(args)]
  content = "\n".join(content_list)
  content += "\n\n請依資料給我一份約200字的分析報告。若有多個策略, \
                  請選出最好的策略及原因, reply in 繁體中文."

  msg = [{
      "role": "system",
      "content": "你是一位專業的證券分析師, 我會給你交易策略的回測績效,\
                  請幫我進行績效分析.不用詳細講解每個欄位, \
                  重點說明即可, 並回答交易策略的好壞"
  }, {
      "role": "user",
      "content": content
  }]

  reply_data = get_reply(msg)
  return reply_data


### 1️⃣3️⃣ 回測結果分析

In [ ]:
stats = ai_backtest(stock_id="2330.TW",
           period="5y",
           user_msg="MACD",
           add_msg="請設置10%的停損點與20%的停利點")
reply = backtest_analysis(stats)
print(reply)


[*********************100%%**********************]  1 of 1 completed


class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.EMA_12, self.data.EMA_26) and self.data.MACD > 0:
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data.EMA_26, self.data.EMA_12) and self.data.MACD < 0:
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2019-07-17 00:00:00
End                       2024-07-17 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   73.623665
Equity Final [$]                   100223.496
Equity Peak [$]                    100229.996
Return [%]                           0.223496
Buy & Hold Return [%]              308.730159
Return (Ann.) [%]                    0.046238
Volatility (Ann.) [%]                0.109217
Sharpe Ratio                         0.423354
Sorti

### 1️⃣4️⃣ 比較多個策略

In [ ]:
# 策略1:MACD+停利停損
stats1 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="MACD",
            add_msg="請設置10%的停損點與20%的停利點")
# 策略2:SMA
stats2 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="SMA",
            add_msg="無")
# 策略3:RSI+停利停損
stats3 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="RSI",
            add_msg="請設置10%的停損點與20%的停利點")

reply = backtest_analysis(stats1, stats2, stats3)
print(reply)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, 0):
        self.buy(size=1,
                    sl=self.data.Close[-1] * 0.90,
                    tp=self.data.Close[-1] * 1.20)
    elif crossover(0, self.data.MACD):
        self.sell(size=1,
                     sl=self.data.Close[-1] * 1.10,
                     tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2019-07-17 00:00:00
End                       2024-07-17 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   73.623665
Equity Final [$]                   100223.496
Equity Peak [$]                    100229.996
Return [%]                           0.223496
Buy & Hold Return [%]              308.730159
Return (Ann.) [%]                    0.046238
Volatility (Ann.) [%]                0.109217
Sharpe Ratio                         0.423354
Sortino Ratio                        0.640277
Calmar 


[*********************100%%**********************]  1 of 1 completed

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.SMA_20, self.data.SMA_50):
        self.buy(size=1)
    elif crossover(self.data.SMA_50, self.data.SMA_20):
        self.sell(size=1)
-----------------------
Start                     2019-07-17 00:00:00
End                       2024-07-17 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   88.249795
Equity Final [$]                    100542.43
Equity Peak [$]                     100567.43
Return [%]                            0.54243
Buy & Hold Return [%]              308.730159
Return (Ann.) [%]                    0.112078
Volatility (Ann.) [%]                0.144124
Sharpe Ratio                         0.777653
Sortino Ratio                        1.159508
Calmar Ratio                         0.295833
Max. Drawdown [%]                   -0.378857
Avg. Drawdown [%]                   -0.032742
Max. Drawdown Duration     1239 days 00:0

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if self.data.RSI[-1] < 30:
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif self.data.RSI[-1] > 70:
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2019-07-17 00:00:00
End                       2024-07-17 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   98.356615
Equity Final [$]                    98567.464
Equity Peak [$]                     100010.52
Return [%]                          -1.432536
Buy & Hold Return [%]              308.730159
Return (Ann.) [%]                    -0.29833
Volatility (Ann.) [%]                0.146811
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. 